## CREANDO UN VOLÚMEN LÓGICO LVM RAID 0

Debemos estar en la máquina virtual con los 5 discos sin formato (si sus discos duros tienen formato, favor restaurar la instantánea apropiada)

https://www.tecnozero.com/servidor/tipos-de-raid-cual-elegir/

Antes de proceder se debe analizar el comando que permite crear el RAID0

lvcreate --type raid0 --stripes 3 --stripesize 4 -n my_lv myvg -L 4G

Al especificar **--type raid0** se crea un volúmen RAID0 sin volúmenes de metadatos. Al especificar un **--type raid0_meta** se crea un volúmen RAID0 con volúmenes de metadatos. Debido a que RAID0 no es resistente, no tiene que almacenar ningún bloque de datos duplicados como RAID1 / 10 o calcular y almacenar ningún bloque de paridad como lo hace RAID4 / 5 / 6. Por lo tanto, no necesita volúmenes de metadatos para mantener el estado sobre el progreso de resincronización de los bloques duplicados o de paridad. Sin embargo, los volúmenes de metadatos se vuelven obligatorios en una conversión de RAID0 a RAID4 / 5 / 6 / 10, y la especificación de raid0_meta preasigna esos volúmenes de metadatos para evitar un fallo de asignación respectivo.

**--stripes 3** especifica la cantidad de dispositivos para distribuir el volúmen lógico, en este caso son 3.

**--stripesize 4** especifica el tamaño de cada franja en kilobytes. Esta es la cantidad de datos que se escriben en un dispositivo antes de pasar al siguiente dispositivo.

**-n my_lv** establece el nombre del raid0 en my_lv.

**myvg** establece el volumen de grupo que se va a utilizar.

**-L 4G** establece el tamaño del volumen raid0.



Nos aseguramos de que tengamos los discos sin formato

`fdisk -l | grep sd`

***
```
root@debian12lvm1:~# fdisk -l | grep sd
Disco /dev/sde: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdf: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdd: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdc: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdb: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sda: 10 GiB, 10737418240 bytes, 20971520 sectores
/dev/sda1  *          2048 18970623 18968576     9G 83 Linux
/dev/sda2         18972670 20969471  1996802   975M  5 Extendida
/dev/sda5         18972672 20969471  1996800   975M 82 Linux swap / Solaris
```
***

Ahora revisamos el script para asegurarse que se formatearan los 5 discos

`cat 123`


***
```
root@debian12lvm1:~# cat 123
#!/bin/bash
#Estos comandos Establecen que van a ser discos msdos master boot record 
parted /dev/sdb mklabel msdos
parted /dev/sdc mklabel msdos
parted /dev/sdd mklabel msdos
parted /dev/sde mklabel msdos
parted /dev/sdf mklabel msdos

#Estos comandos crean una particion primaria y le hacen un set a lvm
parted -s /dev/sdb unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdc unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdd unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sde unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdf unit mib mkpart primary 1 100% set 1 lvm on

# Solo si es RHEL8
#Este comando le dice al kernel que reconozca esto nuevo
#udevadm settle

# De lo contrario ejecute
partprobe -s

```
***


El procedimiento para crear el RAID0 comienza con ejecutar el script 123 para los 5 discos duros

`sh 123`

***
```
root@debian12lvm1:~# sh 123
Information: You may need to update /etc/fstab.

Information: You may need to update /etc/fstab.                           

Information: You may need to update /etc/fstab.                           

Information: You may need to update /etc/fstab.                           

Information: You may need to update /etc/fstab.                           

/dev/sda: msdos partitions 1 2 <5>                                        
/dev/sdb: msdos partitions 1
/dev/sdc: msdos partitions 1
/dev/sdd: msdos partitions 1
/dev/sde: msdos partitions 1
/dev/sdf: msdos partitions 1
```
***


vamos a crear un LVM raid0 de grupo con tres discos

`vgcreate myvg /dev/sdb1  /dev/sdc1  /dev/sdd1`

***
```
root@debian12lvm1:~# vgcreate myvg /dev/sdb1  /dev/sdc1  /dev/sdd1
  Physical volume "/dev/sdb1" successfully created.
  Physical volume "/dev/sdc1" successfully created.
  Physical volume "/dev/sdd1" successfully created.
  Volume group "myvg" successfully created

```
***

Mostramos que el vg group se ha creado satisfactoriamente

`vgs`

***
```
root@debian12lvm1:~# vgs
  VG   #PV #LV #SN Attr   VSize   VFree  
  myvg   3   0   0 wz--n- <14.99g <14.99g
```
***

  
Ahora proseguimos a crear el raid0, crearlo con uan capacidad 

El número tres del comando stripes es porque tenemos tres discos, y stripeSize son bandas de 4 kilobytes

`lvcreate --type raid0 --stripes 3 --stripesize 4 -n my_lv myvg -L 4G`

***
```
root@debian12lvm1:~# lvcreate --type raid0 --stripes 3 --stripesize 4 -n my_lv myvg -L 4G
  Rounding size 4.00 GiB (1024 extents) up to stripe boundary size <4.01 GiB (1026 extents).
  Logical volume "my_lv" created.

```
***

Comprobamos que el volumen se ha creado
`lvs`

***
```
root@debian12lvm1:~# lvs
  LV    VG   Attr       LSize  Pool Origin Data%  Meta%  Move Log Cpy%Sync Convert
  my_lv myvg rwi-a-r--- <4.01g 
```
***


Lo que resta es hacer un filesystem
`mkfs.ext4 /dev/myvg/my_lv`

***
```
root@debian12lvm1:~# mkfs.ext4 /dev/myvg/my_lv
mke2fs 1.47.0 (5-Feb-2023)
Creating filesystem with 1050624 4k blocks and 262944 inodes
Filesystem UUID: da567754-503c-49ad-87ee-e79ba435b0c4
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912, 819200, 884736

Allocating group tables: done                            
Writing inode tables: done                            
Creating journal (16384 blocks): done
Writing superblocks and filesystem accounting information: done
```
***


Ahora lo montamos para comprobar que funciona
`mount /dev/myvg/my_lv /mnt/`


Comprobamos que se ha montado
`df -h /mnt/`

***
```
root@debian12lvm1:~# df -h /mnt/
S.ficheros             Tamaño Usados  Disp Uso% Montado en
/dev/mapper/myvg-my_lv   3.9G    24K  3.7G   1% /mnt
```
***